In [ ]:
#pip install category_encoders

In [131]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)


import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

# Train

__Lectura de datos__

In [51]:
train = pd.read_csv('Train_TP2_Datos_2020-2C.csv')

train = train[(train.Stage == 'Closed Won') | (train.Stage == 'Closed Lost')]
train['Stage'] = train['Stage'].transform(func=lambda x : 1 if x=='Closed Won' else 0)

train = train.drop('Last_Activity', axis=1) #Todos datos nulos
train = train.drop('Actual_Delivery_Date', axis=1) #Todos datos nulos
train = train.drop('Submitted_for_Approval', axis=1) #Todos los datos en cero
train = train.drop('Prod_Category_A', axis=1) #Todos en none
train = train.drop('Sales_Contract_No', axis=1) #Para evitar el problema de target leakage
train = train.drop('Opportunity_Name', axis=1) #Redundante si tenemos Opportunity_ID

***

__FEATURES__

__Aprobación especial de su precio total__

In [52]:
train['Pricing, Delivery_Terms_Quote_Appr'] = train['Pricing, Delivery_Terms_Quote_Appr']\
                                                .transform(func=lambda x : 'Necesita' if x==1 else 'NoNecesita')
train['Pricing, Delivery_Terms_Approved'] = train['Pricing, Delivery_Terms_Approved']\
                                                .transform(func=lambda x : 'Obtuvo' if x==1 else 'NoObtuvo')

train['Aprobacion_Especial'] = train['Pricing, Delivery_Terms_Quote_Appr']+train['Pricing, Delivery_Terms_Approved']

train = train.drop('Pricing, Delivery_Terms_Quote_Appr', axis=1)
train = train.drop('Pricing, Delivery_Terms_Approved', axis=1)

***

__TRF promedio__

In [53]:
train['TRF_promedio'] = train.groupby('Opportunity_ID')['TRF'].transform('mean')
train = train.drop('TRF', axis=1)

***

__Total Taxable Amount (converted)__

In [54]:
#Completo datos faltantes con Total Amount
train['Total_Taxable_Amount'].loc[train.Total_Taxable_Amount==0.00] = train.loc[train.Total_Taxable_Amount==0.00]\
                                            .groupby(['Opportunity_ID'])['Total_Amount'].transform('sum')

#Convierto a una misma moneda (a dólares) con la información que ya viene en ASP, ASP_Currency y ASP_(converted)
toEUR = train[train.ASP_Currency=='EUR'].head(1)[['ASP_(converted)']].iloc[0,0]/train[train.ASP_Currency=='EUR']\
        .head(1)[['ASP']].iloc[0,0]
toAUD = train[train.ASP_Currency=='AUD'].head(1)[['ASP_(converted)']].iloc[0,0]/train[train.ASP_Currency=='AUD']\
        .head(1)[['ASP']].iloc[0,0]
toGBP = train[train.ASP_Currency=='GBP'].head(1)[['ASP_(converted)']].iloc[0,0]/train[train.ASP_Currency=='GBP']\
        .head(1)[['ASP']].iloc[0,0]
toJPY = train[['ASP_(converted)']].iloc[12,0]/train[['ASP']].iloc[12,0]
#En el head(1) de JPY justo habia un nan. Ese numero magico 12 corresponde a la primera fila de moneda JPY con un valor valido

train['Total_Taxable_Amount_(converted)']=0.0
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='EUR'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x*toEUR)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='JPY'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x*toJPY)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='AUD'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x*toAUD)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='GBP'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x*toGBP)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='USD'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x)

train = train.drop('Total_Taxable_Amount_Currency', axis=1)
train = train.drop('Total_Amount_Currency', axis=1)
train = train.drop('Total_Amount', axis=1)
train = train.drop('ASP_Currency', axis=1)
train = train.drop('ASP', axis=1)
train = train.drop('ASP_(converted)_Currency', axis=1)
train = train.drop('ASP_(converted)', axis=1)

***

__Tiempo de entrega (en días)__

In [56]:
#Es la diferencia entre la maxima fecha de entrega (Planned_Delivery_End_Date) y la mínima (Planned_Delivery_Start_Date)
#Reemplazo los Nat de Planned_Delivery_End_Date con los de Start_Date (curiosamente da mejor resultado que
#reemplazarlo con la fecha de Month)
train['Planned_Delivery_End_Date'].loc[train['Planned_Delivery_End_Date']=='NaT'] = train['Planned_Delivery_Start_Date']

train['Planned_Delivery_Start_Date'] = pd.to_datetime(train['Planned_Delivery_Start_Date'])
train['Planned_Delivery_End_Date'] = pd.to_datetime(train['Planned_Delivery_End_Date'])

train['Planned_Delivery_Start_Date_min'] = train.groupby('Opportunity_ID')['Planned_Delivery_Start_Date'].transform('min')
train['Planned_Delivery_End_Date_max'] = train.groupby('Opportunity_ID')['Planned_Delivery_End_Date'].transform('max')

train['Diferencia'] = train['Planned_Delivery_End_Date_max'] - train['Planned_Delivery_Start_Date_min']
train['Diferencia'] = train['Diferencia'].dt.days

train = train.drop('Planned_Delivery_Start_Date', axis=1)
train = train.drop('Planned_Delivery_End_Date', axis=1)
train = train.drop('Planned_Delivery_Start_Date_min', axis=1)
train = train.drop('Planned_Delivery_End_Date_max', axis=1)

***

__Cantidad de productos por oportunidad__

In [57]:
train['Cantidad de productos'] = train.groupby('Opportunity_ID')['ID'].transform('count')
train = train.drop('ID', axis=1)

***

__Elimino lo que ya no voy a usar__

In [58]:
#Elimino columnas que no aportaron para mejorar el resultado
train = train.drop('Bureaucratic_Code_0_Approval', axis=1)
train = train.drop('Bureaucratic_Code_0_Approved', axis=1)
train = train.drop('Account_Created_Date', axis=1)
train = train.drop('Source ', axis=1)
train = train.drop('Billing_Country', axis=1)
train = train.drop('Account_Name', axis=1) #Genera overfitting
train = train.drop('Opportunity_Owner', axis=1)
train = train.drop('Account_Type', axis=1)
train = train.drop('Quote_Type', axis=1)
train = train.drop('Delivery_Terms', axis=1)
train = train.drop('Opportunity_Created_Date', axis=1)
train = train.drop('Product_Type', axis=1)
train = train.drop('Size', axis=1)
train = train.drop('Product_Category_B', axis=1)
train = train.drop('Price', axis=1)
train = train.drop('Currency', axis=1)
train = train.drop('Quote_Expiry_Date', axis=1)
train = train.drop('Last_Modified_Date', axis=1)
train = train.drop('Last_Modified_By', axis=1)
train = train.drop('Product_Family', axis=1)
train = train.drop('Product_Name', axis=1)
train = train.drop('Month', axis=1)
train = train.drop('Delivery_Quarter', axis=1)
train = train.drop('Delivery_Year', axis=1)
train = train.drop('Bureaucratic_Code', axis=1)

#Elimino los duplicados para obtener una oportunidad por fila. Nota: Todos los features que siguen corresponden a
#caracteristicas de las oportunidades, no a caracteristicas especificas de c/producto dentro de una oportunidad
train = train.drop_duplicates('Opportunity_ID')
train = train.drop('Opportunity_ID', axis=1)

***

__CODIFICACIÓN DE VARIABLES CATEGÓRICAS__

__Territorio__

In [59]:
target_enc_terr = ce.TargetEncoder(train['Territory'])
target_enc_terr.fit(train['Territory'], train['Stage'])
train = train.join(target_enc_terr.transform(train['Territory']).add_suffix('_target'))
train = train.drop('Territory', axis=1)

***

__Region__

In [60]:
target_enc_region = ce.TargetEncoder(train['Region'])
target_enc_region.fit(train['Region'], train['Stage'])
train = train.join(target_enc_region.transform(train['Region']).add_suffix('_target'))
train = train.drop('Region', axis=1)

***

__Aprobación especial__

In [61]:
target_enc_AE = ce.TargetEncoder(train['Aprobacion_Especial'])
target_enc_AE.fit(train['Aprobacion_Especial'], train['Stage'])
train = train.join(target_enc_AE.transform(train['Aprobacion_Especial']).add_suffix('_target'))
train = train.drop('Aprobacion_Especial', axis=1)

***

__Marca__

In [62]:
target_enc_brand = ce.TargetEncoder(train['Brand'])
target_enc_brand.fit(train['Brand'], train['Stage'])
train = train.join(target_enc_brand.transform(train['Brand']).add_suffix('_target'))
train = train.drop('Brand', axis=1)

***

__Tipo de oportunidad__

In [63]:
target_enc_Opportunity_Type = ce.TargetEncoder(train['Opportunity_Type'])
target_enc_Opportunity_Type.fit(train['Opportunity_Type'], train['Stage'])
train = train.join(target_enc_Opportunity_Type.transform(train['Opportunity_Type']).add_suffix('_target'))
train = train.drop('Opportunity_Type', axis=1)

***

__Vendedor responsable de la cuenta cliente__

In [64]:
target_enc_Account_Owner = ce.TargetEncoder(train['Account_Owner'])
target_enc_Account_Owner.fit(train['Account_Owner'], train['Stage'])
train = train.join(target_enc_Account_Owner.transform(train['Account_Owner']).add_suffix('_target'))
train = train.drop('Account_Owner', axis=1)

***

In [65]:
train_label = train['Stage']
train_data = train.iloc[:, train.columns != 'Stage']

In [236]:
train_label.value_counts()

1    5072
0    4719
Name: Stage, dtype: int64

***

# Test

__Lectura de datos__

In [66]:
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')

test = test.drop('Last_Activity', axis=1)
test = test.drop('Actual_Delivery_Date', axis=1)
test = test.drop('Submitted_for_Approval', axis=1)
test = test.drop('Prod_Category_A', axis=1)
test = test.drop('Sales_Contract_No', axis=1)
test = test.drop('Opportunity_Name', axis=1)

***

__FEATURES__

__Aprobación especial de su precio total__

In [67]:
test['Pricing, Delivery_Terms_Quote_Appr'] = test['Pricing, Delivery_Terms_Quote_Appr']\
                                                .transform(func=lambda x : 'Necesita' if x==1 else 'NoNecesita')
test['Pricing, Delivery_Terms_Approved'] = test['Pricing, Delivery_Terms_Approved']\
                                                .transform(func=lambda x : 'Obtuvo' if x==1 else 'NoObtuvo')

test['Aprobacion_Especial'] = test['Pricing, Delivery_Terms_Quote_Appr']+test['Pricing, Delivery_Terms_Approved']

test = test.drop('Pricing, Delivery_Terms_Quote_Appr', axis=1)
test = test.drop('Pricing, Delivery_Terms_Approved', axis=1)

***

__TRF promedio__

In [68]:
test['TRF_promedio'] = test.groupby('Opportunity_ID')['TRF'].transform('mean')
test = test.drop('TRF', axis=1)

***

__Total Taxable Amount (converted)__

In [69]:
test['Total_Taxable_Amount'].loc[test.Total_Taxable_Amount==0.00] = test.loc[test.Total_Taxable_Amount==0.00]\
                                            .groupby(['Opportunity_ID'])['Total_Amount'].transform('sum')

test['Total_Taxable_Amount_(converted)']=0.0
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='EUR'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x*toEUR)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='JPY'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x*toJPY)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='AUD'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x*toAUD)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='GBP'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x*toGBP)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='USD'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x)

test = test.drop('Total_Amount_Currency', axis=1)
test = test.drop('Total_Amount', axis=1)
test = test.drop('Total_Taxable_Amount_Currency', axis=1)
test = test.drop('ASP_Currency', axis=1)
test = test.drop('ASP', axis=1)
test = test.drop('ASP_(converted)_Currency', axis=1)
test = test.drop('ASP_(converted)', axis=1)

***

__Tiempo de entrega (en días)__

In [70]:
test['Planned_Delivery_End_Date'].loc[test['Planned_Delivery_End_Date']=='NaT'] = test['Planned_Delivery_Start_Date']

test['Planned_Delivery_Start_Date'] = pd.to_datetime(test['Planned_Delivery_Start_Date'])
test['Planned_Delivery_End_Date'] = pd.to_datetime(test['Planned_Delivery_End_Date'])

test['Planned_Delivery_Start_Date_min'] = test.groupby('Opportunity_ID')['Planned_Delivery_Start_Date'].transform('min')
test['Planned_Delivery_End_Date_max'] = test.groupby('Opportunity_ID')['Planned_Delivery_End_Date'].transform('max')

test['Diferencia'] = test['Planned_Delivery_End_Date_max'] - test['Planned_Delivery_Start_Date_min']
test['Diferencia'] = test['Diferencia'].dt.days

test = test.drop('Planned_Delivery_Start_Date', axis=1)
test = test.drop('Planned_Delivery_End_Date', axis=1)
test = test.drop('Planned_Delivery_Start_Date_min', axis=1)
test = test.drop('Planned_Delivery_End_Date_max', axis=1)

***

__Cantidad de productos por oportunidad__

In [71]:
test['Cantidad de productos'] = test.groupby('Opportunity_ID')['ID'].transform('count')
test = test.drop('ID', axis=1)

***

__Elimino lo que ya no voy a usar__

In [72]:
test = test.drop('Bureaucratic_Code_0_Approval', axis=1)
test = test.drop('Bureaucratic_Code_0_Approved', axis=1)
test = test.drop('Account_Created_Date', axis=1)
test = test.drop('Source ', axis=1)
test = test.drop('Billing_Country', axis=1)
test = test.drop('Account_Name', axis=1)
test = test.drop('Opportunity_Owner', axis=1)
test = test.drop('Account_Type', axis=1)
test = test.drop('Quote_Type', axis=1)
test = test.drop('Delivery_Terms', axis=1)
test = test.drop('Opportunity_Created_Date', axis=1)
test = test.drop('Product_Type', axis=1)
test = test.drop('Size', axis=1)
test = test.drop('Product_Category_B', axis=1)
test = test.drop('Price', axis=1)
test = test.drop('Currency', axis=1)
test = test.drop('Quote_Expiry_Date', axis=1)
test = test.drop('Last_Modified_Date', axis=1)
test = test.drop('Last_Modified_By', axis=1)
test = test.drop('Product_Family', axis=1)
test = test.drop('Product_Name', axis=1)
test = test.drop('Month', axis=1)
test = test.drop('Delivery_Quarter', axis=1)
test = test.drop('Delivery_Year', axis=1)
test = test.drop('Bureaucratic_Code', axis=1)

test = test.drop_duplicates('Opportunity_ID')
IDs = test[['Opportunity_ID']].copy()
test = test.drop('Opportunity_ID', axis=1)

***

__CODIFICACIÓN DE VARIABLES CATEGÓRICAS__

__Territorio__

In [73]:
test = test.join(target_enc_terr.transform(test['Territory']).add_suffix('_target'))
test = test.drop('Territory', axis=1)

***

__Región__

In [74]:
test = test.join(target_enc_region.transform(test['Region']).add_suffix('_target'))
test = test.drop('Region', axis=1)

***

__Aprobación especial__

In [75]:
test = test.join(target_enc_AE.transform(test['Aprobacion_Especial']).add_suffix('_target'))
test = test.drop('Aprobacion_Especial', axis=1)

***

__Marca__

In [76]:
test = test.join(target_enc_brand.transform(test['Brand']).add_suffix('_target'))
test = test.drop('Brand', axis=1)

***

__Tipo de oportunidad__

In [77]:
test = test.join(target_enc_Opportunity_Type.transform(test['Opportunity_Type']).add_suffix('_target'))
test = test.drop('Opportunity_Type', axis=1)

***

__Vendedor responsable de la cuenta cliente__

In [78]:
test = test.join(target_enc_Account_Owner.transform(test['Account_Owner']).add_suffix('_target'))
test = test.drop('Account_Owner', axis=1)

***

# Modelos

__Random Forest__

In [ ]:
modeloRFR=RandomForestRegressor(random_state=0, n_jobs=-1)
LogLoss = make_scorer(log_loss)

__Grid Search Cross Validation__

In [ ]:
parameters = {'n_estimators':[5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100] , 'min_samples_split':[100, 500, 1000]}
gscv = GridSearchCV(modeloRFR, parameters, cv=5, scoring=LogLoss)

In [ ]:
gscv.fit(train_data, train_label)

In [ ]:
print(gscv.best_score_, gscv.best_estimator_)

__Predicción__

In [ ]:
modeloRFR=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=1000,
                      min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=-1,
                      oob_score=False, random_state=0, verbose=0,
                      warm_start=False)
modeloRFR.fit(train_data, train_label)
result = modeloRFR.predict(test)

entregable_RFC = IDs[['Opportunity_ID']].copy()
entregable_RFC['Target'] = result

In [ ]:
entregable_RFC.to_csv('RF_Final.csv', index=False)

In [79]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(train_data, train_label, test_size=0.2, random_state=123)

In [121]:
#INSTANCIO XGBOOST REGRESSOR
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
              min_child_weight = 1, learning_rate = 0.1, gamma = 0,
                subsample= 0.8, colsample_bytree = 0.8, alpha = 10,
                          max_depth = 5, eval_metric=log_loss)
xg_reg.fit(X_train,y_train)

[22:58:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8,
             eval_metric=<function log_loss at 0x00000192683D8D38>, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1,
             objective='binary:logistic', random_state=0, reg_alpha=10,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [155]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(data=dtrain, label=predictors)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
           metrics="logloss", as_pandas=True, early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain, predictors)
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    dtrain_predprob = alg.predict_proba(dtrain)[:,1]
        
    #Print model report:
    print ("\nModel Report")
    rmsle = log_loss(predictors, dtrain_predprob)
    print("RMSLE: %f" % (rmsle))
    print((cvresult["test-logloss-mean"]).tail(1))
                    
   # feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
   # feat_imp.plot(kind='bar', title='Feature Importances')
   # plt.ylabel('Feature Importance Score')

In [273]:
xgb1 = XGBClassifier(
 learning_rate =0.2,
 n_estimators=5000,
 max_depth=8,
 min_child_weight=3,
 gamma=0,
 subsample=0.5,
 colsample_bytree=0.65,
 objective= 'binary:logistic',
 nthread=4,
 seed=27)


In [274]:
modelfit(xgb1, train_data, train_label)

[01:30:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

Model Report
RMSLE: 0.293108
48                   0.38
Name: test-logloss-mean, dtype: float64


In [275]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.2, n_estimators=48, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.5, colsample_bytree=0.65,
 objective= 'binary:logistic', nthread=4, seed=27), 
 param_grid = param_test1, scoring=LogLoss,n_jobs=4,iid=False, cv=5)
gsearch1.fit(train_data,train_label)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

[01:31:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([0.96321411, 0.22599502, 0.21821723, 0.33091373, 0.3159544 ,
         0.31276264, 0.44301429, 0.45278873, 0.42366557, 0.60398283,
         0.52798705, 0.443613  ]),
  'std_fit_time': array([0.37534152, 0.01343411, 0.01105563, 0.00990118, 0.01165477,
         0.00986469, 0.01130841, 0.03973785, 0.02368889, 0.01566387,
         0.01732715, 0.06357095]),
  'mean_score_time': array([0.00578465, 0.00558524, 0.00578318, 0.0057847 , 0.00538568,
         0.00558491, 0.00598416, 0.00618353, 0.00638547, 0.00638318,
         0.00638323, 0.005585  ]),
  'std_score_time': array([7.46773524e-04, 4.88753095e-04, 4.02379075e-04, 7.46148995e-04,
         4.88577656e-04, 4.88967069e-04, 1.90734863e-07, 3.98802796e-04,
         4.89771992e-04, 4.88811327e-04, 4.88772704e-04, 4.88072086e-04]),
  'param_max_depth': masked_array(data=[3, 3, 3, 5, 5, 5, 7, 7, 7, 9, 9, 9],
               mask=[False, False, False, False, False, False, False, False,
                     False, False, F

In [279]:
param_test2 = {
 'max_depth':[1,2,3,4],
 'min_child_weight':[1,2,3,4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.2, n_estimators=48, max_depth=8,
 min_child_weight=1, gamma=0, subsample=0.5, colsample_bytree=0.65,
 objective= 'binary:logistic', nthread=4, seed=27), 
 param_grid = param_test2, scoring=LogLoss,n_jobs=4,iid=False, cv=5)
gsearch2.fit(train_data,train_label)
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

[01:33:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([0.13921905, 0.13443947, 0.13922663, 0.14122214, 0.14361558,
         0.14321656, 0.18709831, 0.18669968, 0.17732491, 0.17692661,
         0.17812247, 0.17253866, 0.22459874, 0.22240462, 0.21781683,
         0.22240481, 0.21741786, 0.21602192, 0.27606101, 0.26788354,
         0.27027645, 0.26828256, 0.2638938 , 0.21063633]),
  'std_fit_time': array([0.00593813, 0.00723006, 0.00425979, 0.00430579, 0.00267637,
         0.00448665, 0.00616688, 0.00277785, 0.00291806, 0.00448651,
         0.00079796, 0.004039  , 0.00390805, 0.00327732, 0.00293128,
         0.00199473, 0.0057463 , 0.00406819, 0.00486143, 0.00430565,
         0.00260062, 0.00189218, 0.00256984, 0.0206272 ]),
  'mean_score_time': array([0.0057847 , 0.00498695, 0.00558529, 0.00538549, 0.00558534,
         0.00558553, 0.00578527, 0.00518665, 0.00478735, 0.00498676,
         0.0051867 , 0.00458746, 0.00498657, 0.00498695, 0.00518627,
         0.00498652, 0.00558519, 0.00498695, 0.00518622, 0.00518627,
  

In [250]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.3, n_estimators=60, max_depth=8,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, seed=27), 
 param_grid = param_test3, scoring=LogLoss,n_jobs=4,iid=False, cv=5)
gsearch3.fit(train_data,train_label)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

[01:19:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([0.59889903, 0.61317549, 0.6017664 , 0.59627757, 0.56197367]),
  'std_fit_time': array([0.00846189, 0.02024256, 0.02285744, 0.01792458, 0.06554538]),
  'mean_score_time': array([0.004004  , 0.01103592, 0.00624843, 0.00937276, 0.00372291]),
  'std_score_time': array([0.00490388, 0.0068879 , 0.00765273, 0.00765283, 0.00606085]),
  'param_gamma': masked_array(data=[0.0, 0.1, 0.2, 0.3, 0.4],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'gamma': 0.0},
   {'gamma': 0.1},
   {'gamma': 0.2},
   {'gamma': 0.3},
   {'gamma': 0.4}],
  'split0_test_score': array([6.50583462, 6.40004603, 6.27663192, 6.41767359, 6.27663029]),
  'split1_test_score': array([7.65579287, 6.72086298, 6.61502322, 7.25006874, 6.96782502]),
  'split2_test_score': array([8.29084861, 8.06152558, 8.16736494, 8.60836504, 8.02624389]),
  'split3_test_score': array([8.20264214, 7.88512244, 8.00860243, 8.06152231, 8.00860366]),

In [251]:
param_test4 = {
 'subsample':[i/10.0 for i in range(4,7)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.3, n_estimators=60, max_depth=8,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4,seed=27), 
 param_grid = param_test4, scoring=LogLoss,n_jobs=4,iid=False, cv=5)
gsearch4.fit(train_data,train_label)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

[01:20:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([0.49587135, 0.52858529, 0.53417077, 0.51163096, 0.5341702 ,
         0.56688323, 0.54554005, 0.57925019, 0.59777937, 0.57486439,
         0.6248529 , 0.58994193]),
  'std_fit_time': array([0.02002001, 0.02333791, 0.01928345, 0.02192302, 0.03074968,
         0.02147932, 0.02372725, 0.01420829, 0.01572677, 0.01169014,
         0.01976016, 0.07073463]),
  'mean_score_time': array([0.00698147, 0.00698171, 0.00678225, 0.00758018, 0.00678234,
         0.00718145, 0.00758028, 0.00797911, 0.00432153, 0.00312438,
         0.00937276, 0.00884156]),
  'std_score_time': array([0.00063098, 0.00109258, 0.00074577, 0.00079794, 0.00074624,
         0.00146552, 0.00149277, 0.00063083, 0.00610721, 0.00624876,
         0.00765283, 0.00602847]),
  'param_colsample_bytree': masked_array(data=[0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8, 0.9, 0.9,
                     0.9],
               mask=[False, False, False, False, False, False, False, False,
                     False, Fals

In [252]:
param_test5 = {
 'subsample':[i/100.0 for i in range(45,70,5)],
 'colsample_bytree':[i/100.0 for i in range(65,80,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.3, n_estimators=60, max_depth=8,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, seed=27), 
 param_grid = param_test5, scoring=LogLoss,n_jobs=4,iid=False, cv=5)
gsearch5.fit(train_data,train_label)
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_

[01:21:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([0.53176813, 0.5940115 , 0.60857105, 0.57885098, 0.56409006,
         0.54334569, 0.55930271, 0.56728206, 0.56169715, 0.56428943,
         0.59221535, 0.62193527, 0.60099187, 0.61415653, 0.56508813]),
  'std_fit_time': array([0.00996674, 0.01397058, 0.01939385, 0.02983056, 0.03035403,
         0.02546145, 0.02168969, 0.02252638, 0.0164898 , 0.0143476 ,
         0.02257013, 0.02762845, 0.03160506, 0.0311218 , 0.10689456]),
  'mean_score_time': array([0.00718102, 0.00818076, 0.00678177, 0.00698175, 0.00718083,
         0.00658236, 0.0071816 , 0.00738039, 0.00817852, 0.00738049,
         0.0067821 , 0.00718126, 0.00678215, 0.00678215, 0.00578485]),
  'std_score_time': array([0.00116297, 0.00132711, 0.00116282, 0.00063068, 0.00116321,
         0.0004885 , 0.00074637, 0.00119709, 0.0013232 , 0.00119694,
         0.00074662, 0.00074658, 0.0009775 , 0.00074657, 0.00146606]),
  'param_colsample_bytree': masked_array(data=[0.65, 0.65, 0.65, 0.65, 0.65, 0.7, 0.7, 0.7, 0.

In [254]:
param_test6 = {
 'reg_alpha':[90, 100,110,150,200]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.3, n_estimators=60, max_depth=8,
 min_child_weight=3, gamma=0, subsample=0.5, colsample_bytree=0.65,
 objective= 'binary:logistic', nthread=4, seed=27), 
 param_grid = param_test6, scoring=LogLoss,n_jobs=4,iid=False, cv=5)
gsearch6.fit(train_data,train_label)
gsearch6.cv_results_, gsearch6.best_params_, gsearch6.best_score_

[01:22:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([0.24833512, 0.23596764, 0.24554315, 0.24155354, 0.2014606 ]),
  'std_fit_time': array([0.00696732, 0.00629535, 0.00868107, 0.00798903, 0.03591979]),
  'mean_score_time': array([0.00598412, 0.00598426, 0.00638342, 0.00578394, 0.0047874 ]),
  'std_score_time': array([5.51978917e-07, 8.91963403e-04, 1.49273774e-03, 3.99422855e-04,
         9.76931437e-04]),
  'param_reg_alpha': masked_array(data=[90, 100, 110, 150, 200],
               mask=[False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'reg_alpha': 90},
   {'reg_alpha': 100},
   {'reg_alpha': 110},
   {'reg_alpha': 150},
   {'reg_alpha': 200}],
  'split0_test_score': array([7.89871798, 8.0045029 , 8.02213494, 8.00450779, 8.44528972]),
  'split1_test_score': array([8.74948323, 8.82004457, 8.76712346, 8.64364306, 9.20813704]),
  'split2_test_score': array([8.27320511, 8.27320307, 8.27320511, 8.36141077, 8.94352375]),
  'split3_test_score': array([8.78475839

In [264]:
param_test7 = {
 'reg_alpha':[i for i in range(520,540,1)]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.3, n_estimators=60, max_depth=8,
 min_child_weight=3, gamma=0, subsample=0.5, colsample_bytree=0.65,
 objective= 'binary:logistic', nthread=4, seed=27), 
 param_grid = param_test7, scoring=LogLoss,n_jobs=4,iid=False, cv=5)
gsearch7.fit(train_data,train_label)
gsearch7.cv_results_, gsearch7.best_params_, gsearch7.best_score_

[01:25:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([0.15757074, 0.15638123, 0.15797682, 0.16057057, 0.15977254,
         0.15697927, 0.15279098, 0.15777845, 0.15318985, 0.15358925,
         0.16136761, 0.15857544, 0.15817642, 0.15498533, 0.1519928 ,
         0.1581768 , 0.15358877, 0.15458636, 0.15638113, 0.13603549]),
  'std_fit_time': array([0.00545929, 0.00609646, 0.00566313, 0.00432428, 0.00385797,
         0.00386755, 0.00612852, 0.00247518, 0.00360145, 0.00367811,
         0.00677642, 0.0071088 , 0.00529985, 0.0031284 , 0.00257055,
         0.00360108, 0.00333768, 0.00520126, 0.00146598, 0.01426715]),
  'mean_score_time': array([0.00518622, 0.0047873 , 0.00478725, 0.00538568, 0.00598421,
         0.00498705, 0.00478749, 0.00478673, 0.00458808, 0.00458794,
         0.00478768, 0.00538592, 0.0057847 , 0.00538578, 0.00518651,
         0.00518684, 0.00518665, 0.00478749, 0.00498724, 0.00478783]),
  'std_score_time': array([7.46059875e-04, 7.46250969e-04, 3.98779101e-04, 1.01724264e-03,
         1.09305893e-03

In [258]:
param_test8 = {
 'reg_lambda':[i for i in range(940,960,1)]
}
gsearch8 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.3, n_estimators=60, max_depth=8,
 min_child_weight=3, gamma=0, subsample=0.5, colsample_bytree=0.65,
 objective= 'binary:logistic', nthread=4, seed=27), 
 param_grid = param_test8, scoring=LogLoss,n_jobs=4,iid=False, cv=5)
gsearch8.fit(train_data,train_label)
gsearch8.cv_results_, gsearch8.best_params_, gsearch8.best_score_

[01:24:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'mean_fit_time': array([0.43663101, 0.45238914, 0.43603296, 0.42785482, 0.46276231,
         0.42905135, 0.42506251, 0.42725668, 0.4238658 , 0.42187147,
         0.42885246, 0.42625942, 0.42785521, 0.42047472, 0.42905183,
         0.42685804, 0.42865324, 0.42406492, 0.42326722, 0.37280216]),
  'std_fit_time': array([0.00765893, 0.03576259, 0.01579937, 0.01562963, 0.03173386,
         0.02171717, 0.01842637, 0.01293605, 0.01330601, 0.0121822 ,
         0.01922561, 0.02027901, 0.0108709 , 0.01432279, 0.02069467,
         0.01753716, 0.01392556, 0.01395527, 0.01732713, 0.08018749]),
  'mean_score_time': array([0.00638323, 0.00618372, 0.00618448, 0.00618405, 0.00658216,
         0.00638332, 0.00618372, 0.00678263, 0.00638285, 0.00598383,
         0.0057848 , 0.00698142, 0.00618343, 0.00558515, 0.00638323,
         0.0065825 , 0.00658216, 0.00817823, 0.00598397, 0.00618367]),
  'std_score_time': array([1.19680565e-03, 3.98945905e-04, 7.46251243e-04, 7.46684487e-04,
         4.88052198e-04

In [174]:
xgb1.fit(X_train,y_train)
preds1 = xgb1.predict(X_test)
preds1_proba = xgb1.predict_proba(X_test)[:,1]
rmsle = log_loss(y_test, preds1_proba)
print("RMSLE: %f" % (rmsle))

[00:06:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RMSLE: 0.388001


In [122]:
preds = xg_reg.predict(X_test)
rmsle = log_loss(y_test, preds)
print("RMSLE: %f" % (rmsle))

RMSLE: 0.390252


In [123]:
data_dmatrix = xgb.DMatrix(data=X,label=y)
params = {"objective":"binary:logistic", "min_child_weight" : 1, "learning_rate":0.1, "gamma" : 0,
                "subsample": 0.8, "colsample_bytree" : 0.8,
                          "max_depth" : 5, "eval_metric":log_loss, "scale_pos_weight" : 1}


In [128]:
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=200, early_stopping_rounds=10,
                    metrics="logloss", as_pandas=True, seed=123)

In [129]:
cv_results.head()

,train-logloss-mean,train-logloss-std,test-logloss-mean,test-logloss-std
0,0.66,0.00,0.66,0.00
1,0.63,0.00,0.63,0.00
2,0.60,0.00,0.61,0.00
3,0.58,0.00,0.58,0.00
4,0.56,0.00,0.56,0.00


In [130]:
print((cv_results["test-logloss-mean"]).tail(1))

158                   0.38
Name: test-logloss-mean, dtype: float64


In [89]:
LogLoss = make_scorer(log_loss)

In [ ]:
print(gscv.best_score_, gscv.best_estimator_)

In [85]:
xg_reg.fit(train_data,train_label)
preds = xg_reg.predict(test)
entregable_RFC = IDs[['Opportunity_ID']].copy()
entregable_RFC['Target'] = preds

In [86]:
entregable_RFC.to_csv('XGBoost_Final.csv', index=False)

In [150]:
xgb1.fit(train_data,train_label)
preds = xgb1.predict_proba(test)[:,1]
entregable_RFC = IDs[['Opportunity_ID']].copy()
entregable_RFC['Target'] = preds

[23:42:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [152]:
entregable_RFC.to_csv('XGBoostClass_Final.csv', index=False)